In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
punch = pd.read_csv('data/punch.csv', header = None)
flex = pd.read_csv('data/flex.csv', header = None)
print(punch)
print(flex)

         0     1     2      3      4      5
0     1.96  0.62  0.94  69.58  98.39  55.54
1     1.84  0.68  0.87  77.94  30.09  55.24
2     1.53  0.71  0.81  73.18 -22.10  40.71
3     1.21  0.71  0.72  52.25 -39.25  19.35
4     0.98  0.69  0.62  20.39 -35.34  -0.92
...    ...   ...   ...    ...    ...    ...
6995 -0.31  0.96 -0.15   0.79   0.18   2.32
6996 -0.36  0.96 -0.13   2.50  -1.04   1.04
6997 -0.36  0.95 -0.14   2.14   0.00   0.49
6998 -0.37  0.95 -0.15   0.43   0.67   1.77
6999 -0.37  0.95 -0.16  -1.34  -1.34   0.67

[7000 rows x 6 columns]
         0     1     2       3       4       5
0    -0.49  1.55 -1.63 -216.99  367.44 -317.70
1    -0.90 -1.98 -1.87 -204.29  170.96 -265.88
2    -1.01 -1.98 -1.87 -214.42  -27.53 -171.76
3    -1.09 -1.98 -1.87 -206.00 -166.39  -93.57
4    -1.00 -1.98 -1.70 -153.81 -160.71  -39.12
...    ...   ...   ...     ...     ...     ...
6995 -0.77  0.62  0.32    9.28    5.92    1.28
6996 -0.84  0.62  0.31   17.33   -4.27   -3.91
6997 -0.90  0.61  0.29  

In [3]:
SAMPLES_PER_GESTURE = 10

In [5]:
def processData(d, v):
    dataX = np.empty([0,SAMPLES_PER_GESTURE*6])
    dataY = np.empty([0])

    data  = d.values
    dataNum = data.shape[0] // SAMPLES_PER_GESTURE
 

    for i in tqdm(range(dataNum)):
        tmp = []
        for j in range(SAMPLES_PER_GESTURE):
            tmp += [(data[i * SAMPLES_PER_GESTURE + j][0] + 2000.0) / 4000.0]
            tmp += [(data[i * SAMPLES_PER_GESTURE + j][1] + 2000.0) / 4000.0]
            tmp += [(data[i * SAMPLES_PER_GESTURE + j][2] + 2000.0) / 4000.0]
            tmp += [(data[i * SAMPLES_PER_GESTURE + j][3] + 2000.0) / 4000.0]
            tmp += [(data[i * SAMPLES_PER_GESTURE + j][4] + 2000.0) / 4000.0]
            tmp += [(data[i * SAMPLES_PER_GESTURE + j][5] + 2000.0) / 4000.0]

        tmp = np.array(tmp)

        tmp = np.expand_dims(tmp, axis = 0)

        dataX = np.concatenate((dataX, tmp), axis = 0)
        dataY = np.append(dataY, v)

    return dataX, dataY

In [6]:
punchX, punchY = processData(punch, 0)
flexX, flexY = processData(flex, 1)
dataX = np.concatenate((punchX, flexX), axis = 0)
dataY = np.concatenate((punchY, flexY), axis = 0)

100%|██████████| 100/100 [00:00<00:00, 1430.34it/s]


In [11]:
permutationTrain = np.random.permutation(dataX.shape[0])
print(permutationTrain)

[ 27 101  53  37 132  58 172 149 114  11 153  26  81 174  13  98 147  65
  95  51  66 110  82  42  67  39 154 104 156 165  88 170 186 119  78  43
  74 145  16  57  50 133  14  61 178 181 179  54 189 125 105  71  20 122
 192 124   2 183 144 198  32  56 151  97  68 155  40 163 107  25   6  10
 171  73 175   0 184 161  24   8  85 102  45 180 109  17  29  44   7  99
 121 103 197 193 130   3 148 166 150 137  12 176 106 138  84  87 113 111
  47  90  31 134  69 152 167  46   4 182  33  92 190 146  15  60  52  35
 129 195 162  55 187  59 177  94  49  76 196  72 164 199  80 131  91 158
  34 126  63  93  19 194  77  79  22 128  28  30 157  38  64 169  41 141
 173 112  70  23 100 139 120  48 108 140 142 188  96  36 127 185 116   1
 159 136 191  75  83 168  18  86  89 135 123 160   9  62 118   5 115  21
 117 143]


In [12]:
dataX = dataX[permutationTrain]
dataY = dataY[permutationTrain]
print(dataY)

[1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1.
 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0.
 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1.
 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1.
 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1.
 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 1.
 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0.
 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1.
 1. 0. 0. 1. 1. 0. 0. 0.]


In [18]:
vfoldSize = int(dataX.shape[0]/100*20)

xTest = dataX[0:vfoldSize]
yTest = dataY[0:vfoldSize]

xTrain = dataX[vfoldSize:dataX.shape[0]]
yTrain = dataY[vfoldSize:dataY.shape[0]]

In [13]:
model = keras.Sequential()
model.add(keras.layers.Dense(32, input_shape =(6*SAMPLES_PER_GESTURE,), activation='relu'))
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(2, activation='softmax'))
adam = keras.optimizers.Adam()
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=adam,
              metrics=['sparse_categorical_accuracy'])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                13472     
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 34        
Total params: 14,034
Trainable params: 14,034
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(xTrain, yTrain, batch_size=1, validation_data=(xTest, yTest), epochs=200, verbose=1)

Train on 160 samples, validate on 40 samples
Epoch 1/200
160/160 [==============================] - 2s 11ms/sample - loss: 0.7294 - sparse_categorical_accuracy: 0.5000 - val_loss: 0.7323 - val_sparse_categorical_accuracy: 0.4000
Epoch 2/200
160/160 [==============================] - 1s 5ms/sample - loss: 0.7042 - sparse_categorical_accuracy: 0.5000 - val_loss: 0.6894 - val_sparse_categorical_accuracy: 0.6000
Epoch 3/200
160/160 [==============================] - 1s 5ms/sample - loss: 0.7047 - sparse_categorical_accuracy: 0.5312 - val_loss: 0.7446 - val_sparse_categorical_accuracy: 0.4000
Epoch 4/200
160/160 [==============================] - 1s 5ms/sample - loss: 0.6994 - sparse_categorical_accuracy: 0.4625 - val_loss: 0.6958 - val_sparse_categorical_accuracy: 0.4000
Epoch 5/200
160/160 [==============================] - 1s 6ms/sample - loss: 0.6929 - sparse_categorical_accuracy: 0.5250 - val_loss: 0.6968 - val_sparse_categorical_accuracy: 0.4000
Epoch 6/200
160/160 [==================

In [13]:
model.save('gs_mnist.h5')